# **Train RF-DETR-N-whole (Nano)**

RF-DETRは、Roboflowが開発したリアルタイムの物体検出モデルです。
このノートブックでは、RF-DETR-N（Nano）モデルをカスタムデータセットでトレーニングします。

参考: https://github.com/roboflow/rf-detr

---

## ⚠️ 使用前の注意

**データの前処理は `train_yolo-seg.ipynb` で行ってください。**

このノートブックは、`train_yolo-seg.ipynb` で準備されたYOLO形式のデータ（`data/train/`）を
COCO形式に変換してRF-DETRでトレーニングします。

### ワークフロー
1. **`train_yolo-seg.ipynb`** でデータの前処理・分割を実行
   - 画像の円形マスク適用
   - ラベルファイル（Fundus, Disc, Macula）のコピー
   - train/valid分割
2. **このノートブック** でYOLO→COCO変換 & RF-DETR-Nトレーニングを実行
   - **Lensアノテーションを自動生成**（画像の内接円）
   - 既存クラスのIDを+1シフト（Fundus:0→1, Disc:1→2, Macula:2→3）
   - 最終的に4クラス（Lens, Fundus, Disc, Macula）で学習

### クラス定義
| Class ID | Class Name | 説明 |
|----------|------------|------|
| 0 | Lens | 画像の内接円（自動生成） |
| 1 | Fundus | 眼底（網膜） |
| 2 | Disc | 視神経乳頭 |
| 3 | Macula | 黄斑 |

※ `data/train/` 内のYOLOデータおよび `data/train_coco/` 内のCOCOデータは、毎回のトレーニングで自動更新されます。

In [3]:
# CUDAが使えるかどうかを確認
import torch

cuda_available = torch.cuda.is_available()

if cuda_available:
    print(f"CUDA is available! GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available.")


CUDA is available! GPU: Quadro RTX 5000


In [4]:
# 環境の確認
import torch
import sys
print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"PyTorch CUDA version: {torch.version.cuda}")


Python version: 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]
PyTorch version: 2.5.1+cu121
CUDA available: True
PyTorch CUDA version: 12.1


In [3]:
# RF-DETRのインストール
# 初回のみ実行してください
# !pip install rfdetr


## **RF-DETR データセット構造**

RF-DETRはCOCO形式のデータセットを使用します。

```
dataset/
├── train/
│   ├── image1.jpg
│   ├── image2.jpg
│   └── _annotations.coco.json
├── valid/
│   ├── image1.jpg
│   └── _annotations.coco.json
```

既存のYOLO形式データをCOCO形式に変換する必要があります。


## **Step 1: データの準備**

train_yolo-seg.ipynbと同様のデータ前処理を行います。


In [5]:
# データディレクトリの設定
import os

# 既存のYOLO形式データのパス
YOLO_DATA_DIR = r'C:\Users\ykita\ROP_AI_project\ROP_project\data\train'

# RF-DETR用のCOCO形式データの出力パス
COCO_DATA_DIR = r'C:\Users\ykita\ROP_AI_project\ROP_project\data\train_coco'

# クラス定義（Lensを追加: 画像の内接円として自動生成）
# Lens: 0, Fundus: 1, Disc: 2, Macula: 3
CLASSES = ['Lens', 'Fundus', 'Disc', 'Macula']
NUM_CLASSES = len(CLASSES)

print(f"YOLO Data Directory: {YOLO_DATA_DIR}")
print(f"COCO Data Directory: {COCO_DATA_DIR}")
print(f"Number of classes: {NUM_CLASSES}")
print(f"Classes: {CLASSES}")

YOLO Data Directory: C:\Users\ykita\ROP_AI_project\ROP_project\data\train
COCO Data Directory: C:\Users\ykita\ROP_AI_project\ROP_project\data\train_coco
Number of classes: 4
Classes: ['Lens', 'Fundus', 'Disc', 'Macula']


In [9]:
# Lens円形ポリゴン生成関数
# 画像の内接円（中心: 画像中央、半径: 短辺の半分）をCOCO形式で生成

import math

def generate_lens_annotation(width, height, annotation_id, image_id, num_points=32):
    """
    画像の内接円としてLensアノテーションを生成
    
    Args:
        width: 画像の幅
        height: 画像の高さ
        annotation_id: アノテーションID
        image_id: 画像ID
        num_points: 円を近似するポリゴンの頂点数（デフォルト32）
    
    Returns:
        dict: COCO形式のアノテーション
    """
    # 円の中心と半径を計算
    center_x = width / 2
    center_y = height / 2
    radius = min(width, height) / 2
    
    # 円をポリゴンで近似（num_points個の頂点）
    segmentation = []
    for i in range(num_points):
        angle = 2 * math.pi * i / num_points
        x = center_x + radius * math.cos(angle)
        y = center_y + radius * math.sin(angle)
        segmentation.extend([x, y])
    
    # バウンディングボックスを計算 [x_min, y_min, width, height]
    x_min = center_x - radius
    y_min = center_y - radius
    bbox_width = radius * 2
    bbox_height = radius * 2
    
    # 円の面積
    area = math.pi * radius * radius
    
    # COCO形式のアノテーション
    annotation = {
        "id": annotation_id,
        "image_id": image_id,
        "category_id": 0,  # Lens = class 0
        "bbox": [x_min, y_min, bbox_width, bbox_height],
        "area": area,
        "segmentation": [segmentation],
        "iscrowd": 0
    }
    
    return annotation

# テスト
test_annotation = generate_lens_annotation(640, 480, 0, 0)
print("=== Lens Annotation Example ===")
print(f"BBox: {test_annotation['bbox']}")
print(f"Area: {test_annotation['area']:.2f}")
print(f"Segmentation points: {len(test_annotation['segmentation'][0]) // 2}")

=== Lens Annotation Example ===
BBox: [80.0, 0.0, 480.0, 480.0]
Area: 180955.74
Segmentation points: 32


In [10]:
# YOLO形式（セグメンテーション）からCOCO形式への変換
# 
# 変更点:
# - 既存のclass_idを+1シフト（Fundus:0→1, Disc:1→2, Macula:2→3）
# - 各画像にLens（class_id=0）を自動追加（画像の内接円）
#
# YOLO Segmentation形式:
#   class_id x1 y1 x2 y2 x3 y3 ... (normalized coordinates)
#
# COCO Detection形式:
#   annotations.json with bounding boxes

import json
import os
from PIL import Image
from tqdm import tqdm
import shutil

def yolo_seg_to_coco(yolo_images_dir, yolo_labels_dir, output_dir, classes, split_name):
    """
    YOLO Segmentation形式からCOCO Detection形式に変換
    
    変更点:
    - 既存アノテーションのclass_idを+1シフト（Lensをclass 0として追加するため）
    - 各画像にLens（class_id=0）アノテーションを自動生成
    
    Args:
        yolo_images_dir: YOLO形式の画像ディレクトリ
        yolo_labels_dir: YOLO形式のラベルディレクトリ
        output_dir: 出力ディレクトリ
        classes: クラスのリスト（Lens, Fundus, Disc, Macula）
        split_name: 'train' or 'valid'
    """
    # 出力ディレクトリの作成
    output_images_dir = os.path.join(output_dir, split_name)
    os.makedirs(output_images_dir, exist_ok=True)
    
    # COCO形式のアノテーション構造
    coco_annotations = {
        "images": [],
        "annotations": [],
        "categories": []
    }
    
    # カテゴリの追加（Lens, Fundus, Disc, Macula）
    for idx, class_name in enumerate(classes):
        coco_annotations["categories"].append({
            "id": idx,
            "name": class_name,
            "supercategory": "object"
        })
    
    annotation_id = 0
    image_id = 0
    
    # 統計情報（クラス別カウント）
    class_counts = {class_name: 0 for class_name in classes}
    
    # 画像ファイルのリストを取得
    image_files = [f for f in os.listdir(yolo_images_dir) 
                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    print(f"Processing {len(image_files)} images for {split_name}...")
    
    for img_file in tqdm(image_files, desc=f"Converting {split_name}"):
        img_path = os.path.join(yolo_images_dir, img_file)
        label_file = os.path.splitext(img_file)[0] + '.txt'
        label_path = os.path.join(yolo_labels_dir, label_file)
        
        # 画像サイズの取得
        try:
            with Image.open(img_path) as img:
                width, height = img.size
        except Exception as e:
            print(f"Error reading image {img_path}: {e}")
            continue
        
        # 画像情報の追加
        coco_annotations["images"].append({
            "id": image_id,
            "file_name": img_file,
            "width": width,
            "height": height
        })
        
        # 画像をコピー
        dst_img_path = os.path.join(output_images_dir, img_file)
        if not os.path.exists(dst_img_path):
            shutil.copy(img_path, dst_img_path)
        
        # === Lensアノテーションを自動追加（class_id=0）===
        lens_annotation = generate_lens_annotation(width, height, annotation_id, image_id)
        coco_annotations["annotations"].append(lens_annotation)
        annotation_id += 1
        class_counts['Lens'] += 1
        
        # === 既存のラベルファイルの処理（class_idを+1シフト）===
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                lines = f.readlines()
            
            for line in lines:
                parts = line.strip().split()
                if len(parts) < 5:  # 少なくともclass_id + 2点（4座標）が必要
                    continue
                
                # 元のclass_idを取得し、+1シフト
                original_class_id = int(parts[0])
                shifted_class_id = original_class_id + 1  # Fundus:0→1, Disc:1→2, Macula:2→3
                
                # クラス名を取得してカウント
                if shifted_class_id < len(classes):
                    class_counts[classes[shifted_class_id]] += 1
                
                # セグメンテーション座標を取得
                coords = list(map(float, parts[1:]))
                
                # ペアになっていない場合はスキップ
                if len(coords) % 2 != 0:
                    continue
                
                # 正規化された座標をピクセル座標に変換
                x_coords = [coords[i] * width for i in range(0, len(coords), 2)]
                y_coords = [coords[i] * height for i in range(1, len(coords), 2)]
                
                # バウンディングボックスを計算
                x_min = min(x_coords)
                y_min = min(y_coords)
                x_max = max(x_coords)
                y_max = max(y_coords)
                
                bbox_width = x_max - x_min
                bbox_height = y_max - y_min
                
                # セグメンテーション（ポリゴン）の作成
                segmentation = []
                for i in range(0, len(coords), 2):
                    segmentation.append(coords[i] * width)
                    segmentation.append(coords[i + 1] * height)
                
                # アノテーションの追加（シフトされたclass_idを使用）
                coco_annotations["annotations"].append({
                    "id": annotation_id,
                    "image_id": image_id,
                    "category_id": shifted_class_id,  # +1シフト済み
                    "bbox": [x_min, y_min, bbox_width, bbox_height],
                    "area": bbox_width * bbox_height,
                    "segmentation": [segmentation],
                    "iscrowd": 0
                })
                annotation_id += 1
        
        image_id += 1
    
    # アノテーションファイルの保存
    annotations_path = os.path.join(output_images_dir, '_annotations.coco.json')
    with open(annotations_path, 'w') as f:
        json.dump(coco_annotations, f, indent=2)
    
    # 結果表示
    print(f"Saved {split_name} annotations to {annotations_path}")
    print(f"  - Images: {len(coco_annotations['images'])}")
    print(f"  - Total Annotations: {len(coco_annotations['annotations'])}")
    print(f"  - Per Class:")
    for class_name in classes:
        print(f"      {class_name}: {class_counts[class_name]}")
    
    return coco_annotations

In [11]:
# YOLO形式からCOCO形式に変換を実行

# 出力ディレクトリのクリア（必要に応じて）
if os.path.exists(COCO_DATA_DIR):
    print(f"Removing existing directory: {COCO_DATA_DIR}")
    shutil.rmtree(COCO_DATA_DIR)
os.makedirs(COCO_DATA_DIR, exist_ok=True)

# Train データの変換
train_yolo_images = os.path.join(YOLO_DATA_DIR, 'images', 'train')
train_yolo_labels = os.path.join(YOLO_DATA_DIR, 'labels', 'train')

print("\n=== Converting Train Data ===")
train_coco = yolo_seg_to_coco(
    yolo_images_dir=train_yolo_images,
    yolo_labels_dir=train_yolo_labels,
    output_dir=COCO_DATA_DIR,
    classes=CLASSES,
    split_name='train'
)

# Valid データの変換
valid_yolo_images = os.path.join(YOLO_DATA_DIR, 'images', 'valid')
valid_yolo_labels = os.path.join(YOLO_DATA_DIR, 'labels', 'valid')

print("\n=== Converting Valid Data ===")
valid_coco = yolo_seg_to_coco(
    yolo_images_dir=valid_yolo_images,
    yolo_labels_dir=valid_yolo_labels,
    output_dir=COCO_DATA_DIR,
    classes=CLASSES,
    split_name='valid'
)

# Testディレクトリを作成（RF-DETRが必要とするため、validデータをコピー）
print("\n=== Creating Test Directory (copy from valid) ===")
test_dir = os.path.join(COCO_DATA_DIR, 'test')
valid_dir = os.path.join(COCO_DATA_DIR, 'valid')
os.makedirs(test_dir, exist_ok=True)

# validディレクトリの内容をtestにコピー
for item in os.listdir(valid_dir):
    src = os.path.join(valid_dir, item)
    dst = os.path.join(test_dir, item)
    if os.path.isfile(src):
        shutil.copy(src, dst)
print(f"Copied valid data to test directory: {test_dir}")

print("\n=== Conversion Complete ===")


Removing existing directory: C:\Users\ykita\ROP_AI_project\ROP_project\data\train_coco

=== Converting Train Data ===
Processing 16479 images for train...


Converting train: 100%|██████████| 16479/16479 [01:18<00:00, 210.01it/s]


Saved train annotations to C:\Users\ykita\ROP_AI_project\ROP_project\data\train_coco\train\_annotations.coco.json
  - Images: 16479
  - Total Annotations: 33302
  - Per Class:
      Lens: 16479
      Fundus: 9875
      Disc: 4701
      Macula: 2247

=== Converting Valid Data ===
Processing 4023 images for valid...


Converting valid: 100%|██████████| 4023/4023 [00:11<00:00, 359.87it/s]


Saved valid annotations to C:\Users\ykita\ROP_AI_project\ROP_project\data\train_coco\valid\_annotations.coco.json
  - Images: 4023
  - Total Annotations: 8881
  - Per Class:
      Lens: 4023
      Fundus: 2364
      Disc: 1329
      Macula: 1165

=== Creating Test Directory (copy from valid) ===
Copied valid data to test directory: C:\Users\ykita\ROP_AI_project\ROP_project\data\train_coco\test

=== Conversion Complete ===


In [12]:
# 変換結果の確認

import os

def count_files_in_directory(directory):
    """指定されたディレクトリ内のファイル数をカウントする"""
    if not os.path.exists(directory):
        return 0
    return len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])

train_dir = os.path.join(COCO_DATA_DIR, 'train')
valid_dir = os.path.join(COCO_DATA_DIR, 'valid')

print("=== COCO Format Data Summary ===")
print(f"Train directory: {train_dir}")
print(f"  Files: {count_files_in_directory(train_dir)}")
print(f"Valid directory: {valid_dir}")
print(f"  Files: {count_files_in_directory(valid_dir)}")


=== COCO Format Data Summary ===
Train directory: C:\Users\ykita\ROP_AI_project\ROP_project\data\train_coco\train
  Files: 16480
Valid directory: C:\Users\ykita\ROP_AI_project\ROP_project\data\train_coco\valid
  Files: 4024


## **Step 2: RF-DETR-N トレーニング**


In [13]:
# RF-DETRのインポートとモデルの初期化
from rfdetr import RFDETRNano

# RF-DETR-Nano モデルの作成
model = RFDETRNano()

print("RF-DETR-Nano model loaded successfully!")


Using a different number of positional encodings than DINOv2, which means we're not loading DINOv2 backbone weights. This is not a problem if finetuning a pretrained RF-DETR model.
Using patch size 16 instead of 14, which means we're not loading DINOv2 backbone weights. This is not a problem if finetuning a pretrained RF-DETR model.
Loading pretrain weights
RF-DETR-Nano model loaded successfully!


In [14]:
# トレーニングパラメータの設定

# トレーニング設定
EPOCHS = 100
BATCH_SIZE = 4  # メモリ不足の場合は2に下げる
LEARNING_RATE = 1e-4
LEARNING_RATE_ENCODER = 1.5e-4
WEIGHT_DECAY = 1e-4
RESOLUTION = 448  # 56と32の公倍数（56×8=448, 32×14=448）

# Early Stopping設定
EARLY_STOPPING = True
EARLY_STOPPING_PATIENCE = 20

# 出力ディレクトリ（4クラスモデル用）
OUTPUT_DIR = r'C:\Users\ykita\ROP_AI_project\ROP_project\runs\rfdetr_4class'

print("=== Training Configuration ===")
print(f"Epochs: {EPOCHS}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Learning Rate: {LEARNING_RATE}")
print(f"Resolution: {RESOLUTION}")
print(f"Output Directory: {OUTPUT_DIR}")

=== Training Configuration ===
Epochs: 100
Batch Size: 4
Learning Rate: 0.0001
Resolution: 448
Output Directory: C:\Users\ykita\ROP_AI_project\ROP_project\runs\rfdetr_4class


In [ ]:
# RF-DETR-N トレーニングの実行

model.train(
    dataset_dir=COCO_DATA_DIR,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    grad_accum_steps=4,
    lr=LEARNING_RATE,
    lr_encoder=LEARNING_RATE_ENCODER,
    weight_decay=WEIGHT_DECAY,
    resolution=RESOLUTION,
    use_ema=True,
    checkpoint_interval=1,  # 毎エポックでチェックポイント保存（best更新時にbest.ptも保存）
    tensorboard=True,
    early_stopping=EARLY_STOPPING,
    early_stopping_patience=EARLY_STOPPING_PATIENCE,
    early_stopping_min_delta=0.001,
    early_stopping_use_ema=True,
    device="cuda",
    output_dir=OUTPUT_DIR
)


Unable to initialize TensorBoard. Logging is turned off for this session.  Run 'pip install tensorboard' to enable logging.
Not using distributed mode
git:
  sha: c9e5f12edab392a6d9c582998bd9ac2b138293fc, status: has uncommited changes, branch: master

Namespace(num_classes=4, grad_accum_steps=4, amp=True, lr=0.0001, lr_encoder=0.00015, batch_size=4, weight_decay=0.0001, epochs=100, lr_drop=100, clip_max_norm=0.1, lr_vit_layer_decay=0.8, lr_component_decay=0.7, do_benchmark=False, dropout=0, drop_path=0.0, drop_mode='standard', drop_schedule='constant', cutoff_epoch=0, pretrained_encoder=None, pretrain_weights='rf-detr-nano.pth', pretrain_exclude_keys=None, pretrain_keys_modify_to_load=None, pretrained_distiller=None, encoder='dinov2_windowed_small', vit_encoder_num_layers=12, window_block_indexes=None, position_embedding='sine', out_feature_indexes=[3, 6, 9, 12], freeze_encoder=False, layer_norm=True, rms_norm=False, backbone_lora=False, force_no_pretrain=False, dec_layers=2, dim_feed

Epoch: [0]  [   0/1029]  eta: 7:25:52  lr: 0.000100  class_error: 78.85  loss: 6.5959 (6.5959)  loss_ce: 1.1630 (1.1630)  loss_bbox: 0.5261 (0.5261)  loss_giou: 0.4949 (0.4949)  loss_ce_0: 1.2069 (1.2069)  loss_bbox_0: 0.4518 (0.4518)  loss_giou_0: 0.4668 (0.4668)  loss_ce_enc: 1.0946 (1.0946)  loss_bbox_enc: 0.5936 (0.5936)  loss_giou_enc: 0.5980 (0.5980)  loss_ce_unscaled: 1.1630 (1.1630)  class_error_unscaled: 78.8462 (78.8462)  loss_bbox_unscaled: 0.1052 (0.1052)  loss_giou_unscaled: 0.2475 (0.2475)  cardinality_error_unscaled: 3183.2500 (3183.2500)  loss_ce_0_unscaled: 1.2069 (1.2069)  loss_bbox_0_unscaled: 0.0904 (0.0904)  loss_giou_0_unscaled: 0.2334 (0.2334)  cardinality_error_0_unscaled: 3318.7500 (3318.7500)  loss_ce_enc_unscaled: 1.0946 (1.0946)  loss_bbox_enc_unscaled: 0.1187 (0.1187)  loss_giou_enc_unscaled: 0.2990 (0.2990)  cardinality_error_enc_unscaled: 3394.2500 (3394.2500)  time: 25.9984  data: 21.7313  max mem: 3131
Epoch: [0]  [  10/1029]  eta: 1:18:44  lr: 0.000100

## **Step 3: 推論テスト（トレーニング後に実行）**

トレーニング完了後、または途中のチェックポイントを使用して推論をテストします。

⚠️ **トレーニングをまだ行っていない場合は、Step 4（トレーニング再開）を先に実行してください。**

Cell 15でチェックポイントのパス（`CHECKPOINT_PATH`）を指定してください。


In [ ]:
# 学習済みモデルのロードと推論（このセルを実行するだけでOK）

import os
import cv2
import torch
import matplotlib.pyplot as plt
from PIL import Image
from rfdetr import RFDETRNano

# ===== 設定 =====
# 4クラスモデルのチェックポイント（トレーニング後に適切なパスを指定）
CHECKPOINT_PATH = r'C:\Users\ykita\ROP_AI_project\ROP_project\runs\rfdetr_4class\checkpoint_best_total.pth'
TEST_IMAGE_PATH = r'C:\Users\ykita\ROP_AI_project\ROP_project\bestimage_validation\2431\images\IMG_2431_0765.jpg'

# 4クラス定義（Lensを追加）
CLASSES = ['Lens', 'Fundus', 'Disc', 'Macula']
THRESHOLD = 0.5

# チェックポイントの確認
print(f"Checkpoint: {CHECKPOINT_PATH}")
print(f"Exists: {os.path.exists(CHECKPOINT_PATH)}")
print(f"Test Image: {TEST_IMAGE_PATH}")
print(f"Exists: {os.path.exists(TEST_IMAGE_PATH)}")
print(f"Classes: {CLASSES}")

# モデルのロード
model = RFDETRNano(pretrain_weights=CHECKPOINT_PATH)
print("\nModel loaded successfully!")

# 推論実行
print(f"\n=== Running Inference (threshold={THRESHOLD}) ===")
detections = model.predict(TEST_IMAGE_PATH, threshold=THRESHOLD)

# 結果の表示（detectionsの形式を確認）
print(f"\nDetection object type: {type(detections)}")
print(f"Detection object: {detections}")

# supervision.Detectionsオブジェクトかどうか確認
# supervision.Detectionsはxyxy, confidence, class_id属性を持つ
parsed_detections = []

if hasattr(detections, 'xyxy') and hasattr(detections, 'confidence') and hasattr(detections, 'class_id'):
    # supervision.Detections形式
    print(f"\n=== supervision.Detections format detected ===")
    print(f"Number of detections: {len(detections.xyxy)}")
    
    for i in range(len(detections.xyxy)):
        bbox = detections.xyxy[i].tolist()  # [x1, y1, x2, y2]
        conf = float(detections.confidence[i]) if detections.confidence is not None else 0.0
        class_id = int(detections.class_id[i]) if detections.class_id is not None else 0
        class_name = CLASSES[class_id] if class_id < len(CLASSES) else f"class_{class_id}"
        
        parsed_detections.append({
            'class_id': class_id,
            'class_name': class_name,
            'confidence': conf,
            'bbox': bbox
        })
        print(f"  [{i+1}] Class: {class_name}, Confidence: {conf:.4f}, BBox: {bbox}")
else:
    # その他の形式（リストやdict）
    print(f"\n=== Other format ===")
    for i, det in enumerate(detections):
        print(f"  Detection {i}: type={type(det)}, value={det}")

print(f"\nTotal parsed detections: {len(parsed_detections)}")

In [ ]:
# 検出結果の可視化（マスク描画版）

import cv2
import matplotlib.pyplot as plt
import numpy as np

# 画像読み込み
img = cv2.imread(TEST_IMAGE_PATH)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_overlay = img.copy()

# カラーマップ（4クラス対応）- RGB形式
# Lens: Yellow, Fundus: Red, Disc: Green, Macula: Blue
colors = [
    (255, 255, 0),  # Lens: Yellow
    (255, 0, 0),    # Fundus: Red
    (0, 255, 0),    # Disc: Green
    (0, 0, 255),    # Macula: Blue
]

# マスク画像を作成
mask_overlay = np.zeros_like(img)

# 検出結果を画像に描画（マスクとしてマージ）
for det in parsed_detections:
    class_id = det['class_id']
    class_name = det['class_name']
    conf = det['confidence']
    bbox = det['bbox']
    color = colors[class_id % len(colors)]
    
    if len(bbox) >= 4:
        x1, y1, x2, y2 = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
        
        # bboxから楕円マスクを作成（眼底・視神経乳頭・黄斑は楕円形に近い）
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2
        axes_x = (x2 - x1) // 2
        axes_y = (y2 - y1) // 2
        
        # マスクオーバーレイに楕円を描画（塗りつぶし）
        cv2.ellipse(mask_overlay, (center_x, center_y), (axes_x, axes_y), 0, 0, 360, color, -1)
        
        # 境界線（楕円の輪郭）
        cv2.ellipse(img_overlay, (center_x, center_y), (axes_x, axes_y), 0, 0, 360, color, 3)
        
        # ラベル描画
        label = f"{class_name}: {conf:.2f}"
        label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        cv2.rectangle(img_overlay, (x1, y1 - label_size[1] - 10), (x1 + label_size[0], y1), color, -1)
        cv2.putText(img_overlay, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

# マスクを元画像に半透明でオーバーレイ
alpha = 0.4
result = cv2.addWeighted(img_overlay, 1, mask_overlay, alpha, 0)

# 画像表示
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(img)
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(mask_overlay)
axes[1].set_title('Mask Overlay')
axes[1].axis('off')

axes[2].imshow(result)
axes[2].set_title(f'Detection with Mask ({len(parsed_detections)} objects)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

# 結果を保存
output_path = r'C:\Users\ykita\ROP_AI_project\ROP_project\runs\rfdetr\inference_result_mask.jpg'
cv2.imwrite(output_path, cv2.cvtColor(result, cv2.COLOR_RGB2BGR))
print(f"\nResult saved to: {output_path}")

### **【参考】旧パイプライン: RT-DETR（Lens検出）→ 円形切り抜き → RF-DETR（Fundus/Disc/Macula検出）**

⚠️ **注意**: このセクションは**旧パイプライン**の参考コードです。

**新しいアプローチ（このノートブック）:**
- RF-DETR単体で4クラス（Lens, Fundus, Disc, Macula）を同時検出

**旧アプローチ（以下のコード）:**
1. **RT-DETR** でLens（レンズ）を検出
2. Lens領域をクロップし、円形マスクを適用
3. **RF-DETR** でFundus/Disc/Maculaを検出

旧パイプラインを使用する場合は、別途RT-DETRモデル（Lens検出用）が必要です。

In [ ]:
# 【参考】旧パイプライン: RT-DETR（Lens検出）→ 円形切り抜き → RF-DETR（Fundus/Disc/Macula検出）
# 
# ⚠️ このコードは旧パイプラインの参考用です。
# 新しいアプローチでは、RF-DETR単体で4クラス（Lens, Fundus, Disc, Macula）を検出します。

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from rfdetr import RFDETRNano
from ultralytics import RTDETR
from tqdm import tqdm

# ===== 設定 =====
# RT-DETRモデル（Lens検出用）- 旧パイプライン用
RTDETR_MODEL_PATH = r'C:\Users\ykita\ROP_AI_project\ROP_project\models\rtdetr-l-1697_1703.pt'

# RF-DETRモデル（Fundus/Disc/Macula検出用）- 旧パイプライン用（3クラスモデル）
RFDETR_CHECKPOINT_PATH = r'C:\Users\ykita\ROP_AI_project\ROP_project\runs\rfdetr\checkpoint0043.pth'

# テスト画像
TEST_IMAGE_PATH = r'C:\Users\ykita\ROP_AI_project\ROP_project\bestimage_validation\2431\images\IMG_2431_0765.jpg'

# パラメータ
GRAY_VALUE = (114, 114, 114)
THRESHOLD = 0.5

# 4クラス定義（新しいアプローチ用）
# 旧パイプラインでは3クラス（Fundus, Disc, Macula）のみ使用
CLASSES_NEW = ['Lens', 'Fundus', 'Disc', 'Macula']  # 新しいアプローチ
CLASSES_OLD = ['Fundus', 'Disc', 'Macula']  # 旧パイプライン

# このセルでは旧パイプラインを使用するためCLASSES_OLDを使用
CLASSES = CLASSES_OLD

# ===== モデルロード =====
print("Loading RT-DETR (Lens detection)...")
rtdetr_model = RTDETR(RTDETR_MODEL_PATH)
print("Loading RF-DETR (Fundus/Disc/Macula detection)...")
rfdetr_model = RFDETRNano(pretrain_weights=RFDETR_CHECKPOINT_PATH)
print("Models loaded!")


def detect_lens_rtdetr(model, image):
    """RT-DETRでLens (cls=0) を検出"""
    results = model(image, verbose=False)
    lens_bbox = None
    
    for r in results:
        if r.boxes is None or len(r.boxes) == 0:
            continue
        for box in r.boxes:
            if int(box.cls) == 0:  # Lens
                lens_bbox = box.xyxy[0].cpu().numpy()
                break
        if lens_bbox is not None:
            break
    
    return lens_bbox


def apply_circular_mask(cropped_image, gray_value=(114, 114, 114)):
    """クロップ画像に円形マスクを適用（Lens外を灰色に）"""
    h, w = cropped_image.shape[:2]
    center_x = w // 2
    center_y = h // 2
    diameter = (w + h) / 2
    radius = int(diameter / 2)
    
    # 円形マスク作成
    circle_mask = np.zeros((h, w), dtype=np.uint8)
    cv2.circle(circle_mask, (center_x, center_y), radius, 255, -1)
    
    # マスク適用
    masked = cropped_image.copy()
    masked[circle_mask == 0] = gray_value
    
    return masked, circle_mask, (center_x, center_y, radius)


def detect_fundus_rfdetr(model, image_or_path, threshold=0.5):
    """RF-DETRでFundus/Disc/Maculaを検出"""
    detections = model.predict(image_or_path, threshold=threshold)
    
    parsed = []
    if hasattr(detections, 'xyxy') and hasattr(detections, 'class_id'):
        for i in range(len(detections.xyxy)):
            bbox = detections.xyxy[i].tolist()
            conf = float(detections.confidence[i]) if detections.confidence is not None else 0.0
            class_id = int(detections.class_id[i]) if detections.class_id is not None else 0
            class_name = CLASSES[class_id] if class_id < len(CLASSES) else f"class_{class_id}"
            parsed.append({
                'class_id': class_id,
                'class_name': class_name,
                'confidence': conf,
                'bbox': bbox
            })
    
    return parsed


def process_pipeline(rtdetr_model, rfdetr_model, image_path, threshold=0.5, gray_value=(114, 114, 114)):
    """
    旧パイプライン：
    1. RT-DETRでLens検出
    2. Lens領域をクロップ + 円形マスク
    3. RF-DETRでFundus/Disc/Macula検出
    """
    # 画像読み込み
    image = cv2.imread(image_path)
    if image is None:
        return None
    
    result = {
        'image_path': image_path,
        'original_image': image.copy(),
        'lens_detected': False,
        'lens_bbox': None,
        'cropped_image': None,
        'masked_image': None,
        'mask_info': None,
        'detections': []
    }
    
    # Stage 1: RT-DETRでLens検出
    lens_bbox = detect_lens_rtdetr(rtdetr_model, image)
    
    if lens_bbox is None:
        print(f"  Warning: Lens not detected in {os.path.basename(image_path)}")
        return result
    
    result['lens_detected'] = True
    result['lens_bbox'] = lens_bbox
    
    # Lens領域をクロップ
    x1, y1, x2, y2 = [int(c) for c in lens_bbox]
    cropped = image[y1:y2, x1:x2]
    
    if cropped.size == 0:
        print(f"  Warning: Empty crop in {os.path.basename(image_path)}")
        return result
    
    result['cropped_image'] = cropped.copy()
    
    # Stage 2: 円形マスク適用
    masked, circle_mask, mask_info = apply_circular_mask(cropped, gray_value)
    result['masked_image'] = masked
    result['mask_info'] = mask_info
    
    # 一時ファイルに保存してRF-DETRで推論
    temp_path = os.path.join(os.path.dirname(image_path), '_temp_masked.jpg')
    cv2.imwrite(temp_path, masked)
    
    # Stage 3: RF-DETRでFundus/Disc/Macula検出
    detections = detect_fundus_rfdetr(rfdetr_model, temp_path, threshold)
    result['detections'] = detections
    
    # 一時ファイル削除
    if os.path.exists(temp_path):
        os.remove(temp_path)
    
    return result


# ===== テスト実行 =====
print(f"\n=== Processing: {TEST_IMAGE_PATH} ===")
result = process_pipeline(rtdetr_model, rfdetr_model, TEST_IMAGE_PATH, threshold=THRESHOLD)

if result and result['lens_detected']:
    print(f"  Lens detected: {result['lens_bbox']}")
    print(f"  Mask info: center=({result['mask_info'][0]}, {result['mask_info'][1]}), radius={result['mask_info'][2]}")
    print(f"\n  Detections ({len(result['detections'])}):")
    for det in result['detections']:
        print(f"    - {det['class_name']}: {det['confidence']:.4f}, bbox={det['bbox']}")
else:
    print("  Lens not detected - pipeline failed")

In [ ]:
# 【参考】旧パイプライン結果の可視化（マスク描画版）

if result and result['lens_detected']:
    # 画像を準備
    original_rgb = cv2.cvtColor(result['original_image'], cv2.COLOR_BGR2RGB)
    cropped_rgb = cv2.cvtColor(result['cropped_image'], cv2.COLOR_BGR2RGB)
    masked_rgb = cv2.cvtColor(result['masked_image'], cv2.COLOR_BGR2RGB)
    
    # 検出結果をマスクで描画
    result_img = masked_rgb.copy()
    mask_overlay = np.zeros_like(masked_rgb)
    
    # カラーマップ（4クラス対応）
    # 旧パイプラインでは3クラスのみ使用（Fundus: Red, Disc: Green, Macula: Blue）
    colors = [
        (255, 255, 0),  # Lens: Yellow（新パイプライン用）
        (255, 0, 0),    # Fundus: Red
        (0, 255, 0),    # Disc: Green
        (0, 0, 255),    # Macula: Blue
    ]
    
    for det in result['detections']:
        class_id = det['class_id']
        class_name = det['class_name']
        conf = det['confidence']
        bbox = det['bbox']
        # 旧パイプラインではclass_idが0から始まるので、+1してcolorsにマッピング
        color = colors[(class_id + 1) % len(colors)]
        
        x1, y1, x2, y2 = [int(v) for v in bbox]
        
        # bboxから楕円マスクを作成
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2
        axes_x = (x2 - x1) // 2
        axes_y = (y2 - y1) // 2
        
        # マスクオーバーレイに楕円を描画（塗りつぶし）
        cv2.ellipse(mask_overlay, (center_x, center_y), (axes_x, axes_y), 0, 0, 360, color, -1)
        
        # 境界線（楕円の輪郭）
        cv2.ellipse(result_img, (center_x, center_y), (axes_x, axes_y), 0, 0, 360, color, 3)
        
        # ラベル描画
        label = f"{class_name}: {conf:.2f}"
        label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        cv2.rectangle(result_img, (x1, y1 - label_size[1] - 10), (x1 + label_size[0], y1), color, -1)
        cv2.putText(result_img, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    # マスクを半透明でオーバーレイ
    alpha = 0.4
    result_with_mask = cv2.addWeighted(result_img, 1, mask_overlay, alpha, 0)
    
    # 4つの画像を表示
    fig, axes = plt.subplots(2, 2, figsize=(14, 14))
    
    # 1. 元画像
    axes[0, 0].imshow(original_rgb)
    axes[0, 0].set_title('1. Original Image')
    axes[0, 0].axis('off')
    
    # 2. Lens領域クロップ
    axes[0, 1].imshow(cropped_rgb)
    axes[0, 1].set_title('2. Lens Cropped (RT-DETR)')
    axes[0, 1].axis('off')
    
    # 3. 円形マスク適用
    axes[1, 0].imshow(masked_rgb)
    axes[1, 0].set_title('3. Circular Mask Applied')
    axes[1, 0].axis('off')
    
    # 4. RF-DETR検出結果（マスク表示）
    axes[1, 1].imshow(result_with_mask)
    axes[1, 1].set_title(f'4. RF-DETR Detection - Mask ({len(result["detections"])} objects)')
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # 保存
    output_path = r'C:\Users\ykita\ROP_AI_project\ROP_project\runs\rfdetr\pipeline_result_mask.jpg'
    cv2.imwrite(output_path, cv2.cvtColor(result_with_mask, cv2.COLOR_RGB2BGR))
    print(f"\nResult saved to: {output_path}")
else:
    print("No result to visualize - Lens not detected")

In [ ]:
# 【オプション】複数画像での推論（別の画像でテストしたい場合）

import os
import glob

def run_inference_on_image(model, image_path, classes, threshold=0.5):
    """単一画像に対して推論を実行し可視化"""
    # 推論
    detections = model.predict(image_path, threshold=threshold)
    
    # 画像読み込み
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # カラーマップ（4クラス対応）
    colors = [
        (255, 255, 0),  # Lens: Yellow
        (255, 0, 0),    # Fundus: Red
        (0, 255, 0),    # Disc: Green
        (0, 0, 255),    # Macula: Blue
    ]
    
    for det in detections:
        class_id = det.get('class_id', det.get('class', 0))
        class_name = classes[class_id] if isinstance(class_id, int) and class_id < len(classes) else str(class_id)
        conf = det.get('confidence', det.get('score', 0))
        bbox = det.get('bbox', det.get('box', []))
        color = colors[class_id % len(colors)] if isinstance(class_id, int) else (255, 255, 0)
        
        if len(bbox) >= 4:
            x1, y1 = int(bbox[0]), int(bbox[1])
            x2, y2 = int(bbox[2]), int(bbox[3])
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)
            label = f"{class_name}: {conf:.2f}"
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
    
    return img, detections

# 別の画像でテスト（パスを変更して実行）
# CLASSES = ['Lens', 'Fundus', 'Disc', 'Macula']  # 4クラス
# test_images = [
#     r'C:\Users\ykita\ROP_AI_project\ROP_project\data\ROP_image\IMG_2025\IMG_2025_0001.jpg',
#     r'C:\Users\ykita\ROP_AI_project\ROP_project\data\ROP_image\IMG_2025\IMG_2025_0002.jpg',
# ]
# 
# for img_path in test_images:
#     if os.path.exists(img_path):
#         img, dets = run_inference_on_image(model, img_path, CLASSES, threshold=0.5)
#         plt.figure(figsize=(12, 8))
#         plt.imshow(img)
#         plt.title(os.path.basename(img_path))
#         plt.axis('off')
#         plt.show()

## **Step 4: トレーニング済みモデルの保存と再開**


In [ ]:
# トレーニングの再開（このセルを実行するだけでOK）

import os
from rfdetr import RFDETRNano

# ===== 設定 =====
COCO_DATA_DIR = r'C:\Users\ykita\ROP_AI_project\ROP_project\data\train_coco'
OUTPUT_DIR = r'C:\Users\ykita\ROP_AI_project\ROP_project\runs\rfdetr_4class'

EPOCHS = 100
BATCH_SIZE = 4
LEARNING_RATE = 1e-4
LEARNING_RATE_ENCODER = 1.5e-4
WEIGHT_DECAY = 1e-4
RESOLUTION = 448

# ===== チェックポイント選択 =====
# 以下から1つ選んでコメントを外してください：
CHECKPOINT_TYPE = "latest"       # 最新のチェックポイント（中断した場所から再開）
# CHECKPOINT_TYPE = "best_ema"   # EMAモデルのベスト
# CHECKPOINT_TYPE = "best_regular"  # 通常モデルのベスト
# CHECKPOINT_TYPE = "best_total"    # 総合ベスト

checkpoint_files = {
    "latest": "checkpoint.pth",
    "best_ema": "checkpoint_best_ema.pth",
    "best_regular": "checkpoint_best_regular.pth",
    "best_total": "checkpoint_best_total.pth"
}
checkpoint_path = os.path.join(OUTPUT_DIR, checkpoint_files[CHECKPOINT_TYPE])

print(f"Checkpoint type: {CHECKPOINT_TYPE}")
print(f"Resuming from: {checkpoint_path}")
print(f"Checkpoint exists: {os.path.exists(checkpoint_path)}")

# モデル作成とトレーニング再開
model = RFDETRNano()
model.train(
    dataset_dir=COCO_DATA_DIR,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    grad_accum_steps=4,
    lr=LEARNING_RATE,
    lr_encoder=LEARNING_RATE_ENCODER,
    weight_decay=WEIGHT_DECAY,
    resolution=RESOLUTION,
    use_ema=True,
    checkpoint_interval=1,
    early_stopping=True,
    early_stopping_patience=20,
    resume=checkpoint_path,
    device="cuda",
    output_dir=OUTPUT_DIR
)

## **補足: RF-DETR モデルバリアント**

RF-DETRには以下のモデルバリアントがあります：

| Model | Parameters | COCO mAP | Speed |
|-------|------------|----------|-------|
| RF-DETR-N (Nano) | ~5M | ~40 | Fastest |
| RF-DETR-B (Base) | ~29M | ~53 | Fast |
| RF-DETR-L (Large) | ~128M | ~56 | Moderate |

他のバリアントを使用する場合：

```python
from rfdetr import RFDETRBase, RFDETRLarge

model = RFDETRBase()  # Base model
model = RFDETRLarge()  # Large model
```


In [ ]:
# モデルのエクスポート（ONNX形式）
# 
# export_path = os.path.join(OUTPUT_DIR, 'rf_detr_nano.onnx')
# model.export(export_path, format='onnx')
# print(f"Model exported to {export_path}")
